**Pump it Up: Data Mining Competition**

**Team: WatrPt**

**Members: Ho Jun Cong Keith (A0184106L), Patt Wei Xuan, Elgin (A0183601M)**


In [1]:
import numpy as np
import pandas as pd
from math import sqrt
import warnings
warnings.filterwarnings('ignore')

In [2]:
url = 'https://raw.githubusercontent.com/keithhjc/IS4242-Project-WatrPt/main/Training%20Set%20Labels.csv?token=AMBVJ2AWO5FQJ7SIXFATVOS7SPYW6'

Y_train = pd.read_csv(url)


In [3]:
url = 'https://raw.githubusercontent.com/keithhjc/IS4242-Project-WatrPt/main/Training%20Set%20Values.csv?token=AMBVJ2GQ6BY5G3A4MYIAXE27SPYUG'

X_train = pd.read_csv(url)


In [4]:
full_set = pd.concat([X_train, Y_train.status_group], axis=1, copy=False)
full_set.head(3)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional


# Part 2: Prescriptive Analytics

In [5]:
!pip install -q pyomo
!apt-get install -y -qq glpk-utils
import pyomo.environ as pe

In [6]:
from pyomo.environ import value

## The Problem in Words

To determine for each Local Government Area (LGA), which non-functional pumps can be replaced at minimum possible cost in order to serve water to everyone.

Input

*    Name of the LGA



Output

*    Names of non-functional pumps to be replaced
*    Total cost

Basically, when an LGA is inputted, we first preprocess the data by:

*    Filter the dataframe according to given LGA 
*    Split the dataframe into two, namely one for functional pumps, and another for non-functional pumps
*    Drop any duplicated pumps with the same longitude, latitude and wpt_name
*    For pumps with the same wpt_name but different longitude and latitude, name them as their original name + a postfix number (Dupe -> Dupe1)
*    Using the non-functional pump dataframe from step 2, for each pump, identify the nearest pump and append its name, population it can support, and cost it takes to transport



## Data Preprocessing

We start by removing records that

1.   Don't have location information (longitude or latitude == 0)
2.   Have pumps with 0 surrounding population (population != 0)
3.   Have no name for the pump (wpt_name != none)



In [7]:
part_2_set = full_set[['amount_tsh','longitude', 'latitude', 'wpt_name', 'lga', 'population', 'status_group']]
part_2_set = part_2_set[part_2_set['longitude'] !=0]
part_2_set = part_2_set[part_2_set['latitude'] !=0]
part_2_set = part_2_set[part_2_set['population'] !=0]
part_2_set = part_2_set[part_2_set['wpt_name'] != 'none']
part_2_set

,amount_tsh,longitude,latitude,wpt_name,lga,population,status_group
1,0.0,34.698766,-2.147466,Zahanati,Serengeti,280,functional
2,25.0,37.460664,-3.821329,Kwa Mahundi,Simanjiro,250,functional
3,0.0,38.486161,-11.155298,Zahanati Ya Nanyumbu,Nanyumbu,58,non functional
5,20.0,39.172796,-4.765587,Tajiri,Mkinga,1,functional
10,0.0,39.209518,-7.034139,Mzee Hokororo,Mkuranga,345,functional
...,...,...,...,...,...,...,...
59391,0.0,38.044070,-4.272218,Kwa,Same,210,non functional
59394,500.0,37.634053,-6.124830,Chimeredya,Mvomero,89,non functional
59395,10.0,37.169807,-3.253847,Area Three Namba 27,Hai,125,functional
59396,4700.0,35.249991,-9.070629,Kwa Yahona Kuvala,Njombe,56,functional


Next we take a sample input to build our model and continue data preprocessing.

The input can be any valid LGA.

In [8]:
##input the name of the LGA
input = 'Nanyumbu'
lga_set = part_2_set[part_2_set['lga'] == input]

We then continue preprocessing by first generating the construction cost to repair each pump, since the value is static and decided by TSH

In [9]:
##generate and preprocess construction cost for each pump
lga_set['construction_cost'] = (lga_set['amount_tsh'] * 0.05) + 100
lga_set = lga_set.drop(columns=['amount_tsh'])

We seperate the functional pumps from the non-functional pumps to make the data easier to work with

In [10]:
lga_functional = lga_set[lga_set['status_group'] == 'functional']
lga_non_functional = lga_set[lga_set['status_group'] == 'non functional']

For the functional set, 

*   We first ensure that no pumps are duplicated (duplicated pumps are defined by pumps with the same longitude, latitude and name)

*   We then rename any pumps with the same name, but different longitude and latitude since we need unique values for the pump names

In [11]:
##Drop Duplicated pumps with the same longitude, latitude and wpt_name
lga_functional = lga_functional.drop_duplicates(subset=['longitude', 'latitude', 'wpt_name'], keep='last')

##Replace Duplicated pumps with same name but different longitude/latitude with Name + postfix number
lga_functional['wpt_name'] = lga_functional['wpt_name'] + lga_functional.groupby(['wpt_name']).cumcount().astype(str).replace('0','')

lga_functional = lga_functional.reset_index().drop(columns=['index'])
lga_functional

,longitude,latitude,wpt_name,lga,population,status_group,construction_cost
0,38.242246,-11.136034,Kwa Mustapha Rashidi,Nanyumbu,86,functional,100.00
1,38.358985,-11.243879,Lukwika Camp 2,Nanyumbu,56,functional,100.00
2,38.721621,-10.831273,Nanyomani,Nanyumbu,56,functional,100.00
3,38.644966,-10.829329,Juwamaki 2,Nanyumbu,130,functional,100.00
4,38.511580,-11.150792,Shuleni Mkuula,Nanyumbu,45,functional,100.00
5,38.529403,-11.070651,Kwa Salumu Hassani,Nanyumbu,99,functional,100.00
6,38.340059,-10.935739,Kwa Sina,Nanyumbu,150,functional,100.00
7,38.367407,-10.922912,Halmashauri,Nanyumbu,30,functional,102.50
8,38.413291,-11.024038,Kwa Omary Malki,Nanyumbu,140,functional,102.50
9,38.164017,-11.206286,Lukwika Game Reserve,Nanyumbu,15,functional,100.00


For the non-functional set, we repeat the same process: 

*   We first ensure that no pumps are duplicated (duplicated pumps are defined by pumps with the same longitude, latitude and name)

*   We then rename any pumps with the same name, but different longitude and latitude since we need unique values for the pump names

In [12]:
##Drop Duplicated pumps with the same longitude, latitude and wpt_name
lga_non_functional = lga_non_functional.drop_duplicates(subset=['longitude', 'latitude', 'wpt_name'], keep='last')

##Replace Duplicated pumps with same name but different longitude/latitude with Name + postfix number
lga_non_functional['wpt_name'] = lga_non_functional['wpt_name'] + lga_non_functional.groupby(['wpt_name']).cumcount().astype(str).replace('0','')

lga_non_functional = lga_non_functional.reset_index(drop=True)
lga_non_functional

,longitude,latitude,wpt_name,lga,population,status_group,construction_cost
0,38.486161,-11.155298,Zahanati Ya Nanyumbu,Nanyumbu,58,non functional,100.0
1,38.691033,-10.876661,Kwa Zena Rajabu 2,Nanyumbu,36,non functional,100.0
2,38.117806,-10.925890,Namajani,Nanyumbu,85,non functional,102.5
3,38.119977,-10.926111,Kwa Peter,Nanyumbu,140,non functional,100.0
4,38.527007,-11.164013,Mission,Nanyumbu,99,non functional,100.0
...,...,...,...,...,...,...,...
111,38.428913,-10.931789,Kwa Chilumba Jabiri,Nanyumbu,100,non functional,101.5
112,38.166363,-10.925159,Hospitali,Nanyumbu,56,non functional,100.0
113,38.778164,-10.929034,Kwa Bonga,Nanyumbu,48,non functional,100.0
114,38.416050,-11.024982,Shuleni4,Nanyumbu,1,non functional,100.0


For every non-functional pump in the LGA, we then:
 

1.   Generate the cost it takes to transport water from the nearest functional pump in the LGA
2.   Append the name, population and cost to transport from that pump.

 

In [13]:
##Append name, population, and cost to transport
all_transport_pump = []
funct_pump_name = []
funct_pump_population = []
funct_pump_cost = []
cost = 999999999
population = "0"
wpt_name = "default"
for ind in lga_non_functional.index:
  longitude = lga_non_functional['longitude'][ind]
  latitude = lga_non_functional['latitude'][ind]
  population = lga_non_functional['population'][ind]
  for ind in lga_functional.index:
    funct_longitude = lga_functional['longitude'][ind]
    funct_latitude = lga_functional['latitude'][ind]
    current_cost = 100 + 2000*(sqrt((latitude - funct_latitude)**2 + (longitude - funct_longitude)**2))
    if current_cost < cost:
      wpt_name = lga_functional['wpt_name'][ind]
      population = lga_functional['population'][ind]
      cost = current_cost
  funct_pump_name.append(wpt_name)
  funct_pump_population.append(population)
  funct_pump_cost.append(cost)
  cost = 999999999
lga_non_functional['funct_pump_name'] = funct_pump_name
lga_non_functional['funct_pump_population'] = funct_pump_population
lga_non_functional['funct_pump_cost'] = funct_pump_cost

lga_non_functional

,longitude,latitude,wpt_name,lga,population,status_group,construction_cost,funct_pump_name,funct_pump_population,funct_pump_cost
0,38.486161,-11.155298,Zahanati Ya Nanyumbu,Nanyumbu,58,non functional,100.0,Kwa Emmanuel Mkwemba,145,111.698356
1,38.691033,-10.876661,Kwa Zena Rajabu 2,Nanyumbu,36,non functional,100.0,Kwa Zena Rajabu,86,100.495139
2,38.117806,-10.925890,Namajani,Nanyumbu,85,non functional,102.5,Shuleni,56,343.670861
3,38.119977,-10.926111,Kwa Peter,Nanyumbu,140,non functional,100.0,Shuleni,56,339.589741
4,38.527007,-11.164013,Mission,Nanyumbu,99,non functional,100.0,Shuleni Mkuula,45,140.633776
...,...,...,...,...,...,...,...,...,...,...
111,38.428913,-10.931789,Kwa Chilumba Jabiri,Nanyumbu,100,non functional,101.5,Kwa Chirumba,130,175.069332
112,38.166363,-10.925159,Hospitali,Nanyumbu,56,non functional,100.0,Shuleni,56,251.230448
113,38.778164,-10.929034,Kwa Bonga,Nanyumbu,48,non functional,100.0,Kwa Mlola,56,153.990937
114,38.416050,-11.024982,Shuleni4,Nanyumbu,1,non functional,100.0,Kwa Omary Malki,140,105.833089


## Our Model

We start by defining our ConcreteModel and converting our data into dictionaries and Decision variables. 

## Decision Variables & Objective Function


Total cost to: 
1. Replace non-functional pump => **[100 + 0.05(TSH)]**


OR 
2. Transport water to non-functional pump => **[100 + 2000(√[(Latitude - Latitude1)² + (Longitude - Longitude1)²]** multiplied by the number of functional pumps required


For each pump,
**Cost = X[100 + 0.05(TSH)] + (1-X)([100 + 2000(√[(Latitude1 - Latitude2)² + (Longitude1 - Longitude2)²])**

Where X is either 0 or 1, 0 for water transport, and 1 for repair

The decision variable in our code is represented as **pump.to_construct**, a value that can take a Boolean value of 0 or 1 and is used to represent whether a pump should be repaired or transported water.

In [14]:
pump = pe.ConcreteModel()

wpt_names = lga_non_functional["wpt_name"].tolist()
pump.to_construct = pe.Var(wpt_names, domain=pe.Boolean)

We then generate a few key value pairs to pair up our decision variable with other parameters that we will use in building our constraints

In [15]:
construct_costs = {nfpump: price for nfpump, price in zip(wpt_names, lga_non_functional["construction_cost"].tolist())}
transport_costs = {nfpump: price for nfpump, price in zip(wpt_names, lga_non_functional["funct_pump_cost"].tolist())}
populations = {nfpump: pop for nfpump, pop in zip(wpt_names, lga_non_functional["population"].tolist())}
functioning_populations = {nfpump: pop for nfpump, pop in zip(wpt_names, lga_non_functional["funct_pump_population"].tolist())}

We define our objective as the total cost it takes to either repair the pump or transport water from the nearest functional pump.

The preprocessing of the data in the earlier stage allows us to easily access the name, population and cost to build to the nearest functional pump.

In building the model, we first start by defining the decision variables for the model to choose. In this case, our decision variables is a **boolean value** that can either evaluate to 0 or 1, one for each pump.

A pump with a decision variable value of 1 should be repaired and a pump with a decision variable value of 0 should have water transported from the nearest functional pump.

This is reflected in the objective statement, as the statement minimizes the cost and only adds either the cost of the pump repair or the cost of the water transport, depending on the value of the **to_construct** decision variable.

In [16]:
pump.obj = pe.Objective(expr=sum(construct_costs[i]*pump.to_construct[i] for i in wpt_names) 
+ sum(transport_costs[i] * (1 - pump.to_construct[i]) for i in wpt_names), sense=pe.minimize)

## Our Non-Numerical Constraints

1. When a pump is functional, it can serve non-functional pumps around it. 

2. Non-functional pumps can only be served by the nearest pump next to it.

3. We do not need to account for a pump serving other pumps when it is functional from being repaired.

##Our Numerical Constraints

1. A functional pump can serve one or many non-functional pumps, as long as it has the population capacity to do so.

In [17]:
def population_rule(pump):
  for name in wpt_names:
    if construct_costs[name] > transport_costs[name]:
      if populations[name] > functioning_populations[name]:
        return pe.Constraint.Infeasible
  return pe.Constraint.Feasible

pump.population_con = pe.Constraint(rule=population_rule)

pump.pprint()

1 Set Declarations
    to_construct_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :  116 : {'Zahanati Ya Nanyumbu', 'Kwa Zena Rajabu 2', 'Namajani', 'Kwa Peter', 'Mission', 'Kwa Chikwalo', 'Minazini', 'Kwa Ally Omary', 'Nandete', 'Kwa Rehema Athumani', 'Kwa Ally Mohamed', 'Shuleni', 'Kwa Kalha', 'Kwa Ghohoni', 'Kwa Yusuph Namoyo', 'Kwa Kaluma', 'Kwa Hamza 3', 'Kwa Ismail', 'Mkalia', 'Likuku', 'Kwa Rumeni 2', 'Namatunu 1', 'Kwa Mbanga', 'Mkoroshoni', 'Kwa Tindwa', 'Kwa Masisa', 'Shuleni1', 'Kwa Mtuta', 'Kwa Mwenyemaviha', 'Demuni', 'Kwa Wiliam', 'Kwa Sitta', 'Kwa Rumeni', 'Kwa Ddwani Mteule', 'Kwa Yassin', 'Kwa Kaweni Japhari', 'Lukwika Camp', 'Mkoroshoni 2', 'Shuleni2', 'Kwa Mwanahafa Rajabu', 'Kwa Nihiha', 'Shule Ya Msingi Chungu', 'Kwa Alukama 2', 'Kwa Kalambwa', 'Kwa Salimu Ally', 'Kwa Lilonde', 'Ngangasimiche', 'Kwa Mohamed Kantinda', 'Kwa Adamu Imelale', 'Kwa Chilala 2', 'Kwa Mohamed Mole', 'Nandete1', 'K

The output is the minimal total cost it takes to either repair or transport water to each pump, as well as a list of boolean values, 1 if the pump is to be repaired and 0 if the pump is to be transported water.

In [18]:
opt = pe.SolverFactory('glpk', executable='/usr/bin/glpsol')
results = opt.solve(pump)

print('\nCost = ', pump.obj())
pump.display()


Cost =  11622.0
Model unknown

  Variables:
    to_construct : Size=116, Index=to_construct_index
        Key                    : Lower : Value : Upper : Fixed : Stale : Domain
                    Barabarani :     0 :   1.0 :     1 : False : False : Boolean
                         Bosco :     0 :   1.0 :     1 : False : False : Boolean
                        Demuni :     0 :   1.0 :     1 : False : False : Boolean
                      Demuni 2 :     0 :   1.0 :     1 : False : False : Boolean
                     Hospitali :     0 :   1.0 :     1 : False : False : Boolean
                      Kulumero :     0 :   1.0 :     1 : False : False : Boolean
                Kwa Abeda Issa :     0 :   1.0 :     1 : False : False : Boolean
             Kwa Adamu Imelale :     0 :   1.0 :     1 : False : False : Boolean
              Kwa Ally Mohamed :     0 :   1.0 :     1 : False : False : Boolean
                Kwa Ally Omary :     0 :   1.0 :     1 : False : False : Boolean
           